In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['movies_metadata.csv', 'ratings.csv', 'links_small.csv', 'links.csv', 'credits.csv', 'ratings_small.csv', 'keywords.csv']


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast 
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, evaluate

import warnings; warnings.simplefilter('ignore')


In [3]:
credits = pd.read_csv('../input/credits.csv')
keywords = pd.read_csv('../input/keywords.csv')
links_small = pd.read_csv('../input/links_small.csv')
md = pd.read_csv('../input/movies_metadata.csv')
ratings = pd.read_csv('../input/ratings_small.csv')

In [4]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [5]:
credits.columns

Index(['cast', 'crew', 'id'], dtype='object')

In [6]:
credits.shape


(45476, 3)

In [7]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
cast    45476 non-null object
crew    45476 non-null object
id      45476 non-null int64
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [8]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [9]:
keywords.columns


Index(['id', 'keywords'], dtype='object')

In [10]:
keywords.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
id          46419 non-null int64
keywords    46419 non-null object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


In [11]:
links_small.columns

Index(['movieId', 'imdbId', 'tmdbId'], dtype='object')

In [12]:
md.iloc[0:3].transpose()

,0,1,2
adult,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN,"{'id': 119050, 'name': 'Grumpy Old Men Collect..."
budget,30000000,65000000,0
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
homepage,http://toystory.disney.com/toy-story,NaN,NaN
id,862,8844,15602
imdb_id,tt0114709,tt0113497,tt0113228
original_language,en,en,en
original_title,Toy Story,Jumanji,Grumpier Old Men
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...


# Simple recommendation system

In [13]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i[
    'name'] for i in x] if isinstance(x, list) else [])

In [14]:
# this is V
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')

# this is R
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')

# this is C
C = vote_averages.mean()
C

5.244896612406511

In [15]:

m = vote_counts.quantile(0.95)
m

434.0

In [16]:
# Pre-processing step for getting year from date by splliting it using '-'

md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(
    lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [17]:
qualified = md[(md['vote_count'] >= m) & 
               (md['vote_count'].notnull()) & 
               (md['vote_average'].notnull())][['title', 
                                                'year', 
                                                'vote_count', 
                                                'vote_average', 
                                                'popularity', 
                                                'genres']]

qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

In [18]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [19]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [20]:

qualified = qualified.sort_values('wr', ascending=False).head(250)

In [21]:
qualified.head(15)

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.1081,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.2135,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.8696,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.0707,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.95,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.6454,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.3244,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.3072,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.4235,"[Adventure, Fantasy, Action]",7.851924


In [22]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)
gen_md.head(3).transpose()

,0,0,0
adult,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...","{'id': 10194, 'name': 'Toy Story Collection', ...","{'id': 10194, 'name': 'Toy Story Collection', ..."
budget,30000000,30000000,30000000
homepage,http://toystory.disney.com/toy-story,http://toystory.disney.com/toy-story,http://toystory.disney.com/toy-story
id,862,862,862
imdb_id,tt0114709,tt0114709,tt0114709
original_language,en,en,en
original_title,Toy Story,Toy Story,Toy Story
overview,"Led by Woody, Andy's toys live happily in his ...","Led by Woody, Andy's toys live happily in his ...","Led by Woody, Andy's toys live happily in his ..."
popularity,21.9469,21.9469,21.9469


In [23]:
# Generate Chart
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & 
                   (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: 
                        (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C),
                        axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [24]:
build_chart('Romance').head(15)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,8.565285
351,Forrest Gump,1994,8147,8,48.3072,7.971357
876,Vertigo,1958,1162,8,18.2082,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.8451,7.745154
1132,Cinema Paradiso,1988,834,8,14.177,7.744878
19901,Paperman,2012,734,8,7.19863,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.9943,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166


# Content based recommendation system¶


In [25]:
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [26]:

## Pre-processing step

def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [27]:
md['id'] = md['id'].apply(convert_int)
md[md['id'].isnull()]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[Carousel Productions, Vision View Entertainme...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",NaN,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[Aniplex, GoHands, BROSTA TV, Mardock Scramble...","[{'iso_3166_1': 'US', 'name': 'United States o...",NaN,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[Odyssey Media, Pulser Productions, Rogue Stat...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",NaN,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [28]:
md = md.drop([19730, 29503, 35587])


In [29]:
md['id'] = md['id'].astype('int')


In [30]:

smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

Content based recommendation system: : Using movie description and taglines

In [31]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [32]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [33]:
tfidf_matrix.shape

(9099, 268124)

In [34]:
# http://scikit-learn.org/stable/modules/metrics.html#linear-kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [35]:
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

In [36]:
df_cos=pd.DataFrame(linear_kernel(tfidf_matrix, tfidf_matrix))
df_cos.head()

,0,1,2,3,4,5,6,7,8,9,...,9089,9090,9091,9092,9093,9094,9095,9096,9097,9098
0,1.000000,0.006805,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.005670,0.000000,0.000000,0.000000,0.000000,0.004780,0.000000,0.003449,0.000000
1,0.006805,1.000000,0.015311,0.0,0.002237,0.014747,0.000000,0.000000,0.033137,0.0,...,0.000000,0.006692,0.000000,0.000000,0.000000,0.001748,0.003671,0.003571,0.007623,0.000000
2,0.000000,0.015311,1.000000,0.0,0.007899,0.000000,0.000000,0.002349,0.000000,0.0,...,0.000000,0.000000,0.000000,0.002311,0.001926,0.002209,0.000000,0.000000,0.002865,0.004722
3,0.000000,0.000000,0.000000,1.0,0.000000,0.002152,0.000000,0.006528,0.000000,0.0,...,0.016816,0.000000,0.003570,0.001992,0.000000,0.000000,0.000000,0.000000,0.008673,0.000000
4,0.000000,0.002237,0.007899,0.0,1.000000,0.000000,0.008263,0.000000,0.011411,0.0,...,0.000000,0.000000,0.006026,0.003362,0.000000,0.003800,0.001869,0.004372,0.001388,0.000000


In [37]:
index_to_movie_id = ratings['movieId']
df_cos.columns = [str(index_to_movie_id[int(col)]) for col in df_cos.columns]
df_cos.index = [index_to_movie_id[idx] for idx in df_cos.index]
df_cos.head()

,31,1029,1061,1129,1172,1263,1287,1293,1339,1343,...,919,1136,1148,1193,1213,1221,1356,1673,1722,1923
31,1.000000,0.006805,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.005670,0.000000,0.000000,0.000000,0.000000,0.004780,0.000000,0.003449,0.000000
1029,0.006805,1.000000,0.015311,0.0,0.002237,0.014747,0.000000,0.000000,0.033137,0.0,...,0.000000,0.006692,0.000000,0.000000,0.000000,0.001748,0.003671,0.003571,0.007623,0.000000
1061,0.000000,0.015311,1.000000,0.0,0.007899,0.000000,0.000000,0.002349,0.000000,0.0,...,0.000000,0.000000,0.000000,0.002311,0.001926,0.002209,0.000000,0.000000,0.002865,0.004722
1129,0.000000,0.000000,0.000000,1.0,0.000000,0.002152,0.000000,0.006528,0.000000,0.0,...,0.016816,0.000000,0.003570,0.001992,0.000000,0.000000,0.000000,0.000000,0.008673,0.000000
1172,0.000000,0.002237,0.007899,0.0,1.000000,0.000000,0.008263,0.000000,0.011411,0.0,...,0.000000,0.000000,0.006026,0.003362,0.000000,0.003800,0.001869,0.004372,0.001388,0.000000


In [38]:
df_cos.iloc[0].sort_values(ascending=False)[:10]

31        1.000000
103228    0.248068
8376      0.230813
1198      0.127272
111759    0.101640
595       0.080479
34326     0.072698
1         0.071859
260       0.059213
920       0.056647
Name: 31, dtype: float64

In [39]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [40]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [41]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [42]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

# Content based RS : Using movie description, taglines, keywords, cast, director and genres

In [43]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [44]:
md.shape

(45463, 25)

In [45]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [46]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 28)

In [47]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [48]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [49]:

smd['director'] = smd['crew'].apply(get_director)
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [50]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

In [51]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

In [52]:
s = s[s > 1]

In [53]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [54]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [55]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [56]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [57]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [58]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [59]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [60]:
get_recommendations('The Dark Knight').head(10)

8031         The Dark Knight Rises
6218                 Batman Begins
6623                  The Prestige
2085                     Following
7648                     Inception
4145                      Insomnia
3381                       Memento
8613                  Interstellar
7659    Batman: Under the Red Hood
1134                Batman Returns
Name: title, dtype: object

In [61]:

get_recommendations('Inception').head(10)

6623                             The Prestige
3381                                  Memento
4145                                 Insomnia
2085                                Following
8031                    The Dark Knight Rises
8613                             Interstellar
6981                          The Dark Knight
6218                            Batman Begins
5638    Sky Captain and the World of Tomorrow
8500                                  Don Jon
Name: title, dtype: object

In [62]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & 
                       (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [63]:
improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,year,wr
7648,Inception,14075,8,2010,7.917588
8613,Interstellar,11187,8,2014,7.897107
6623,The Prestige,4510,8,2006,7.758148
3381,Memento,4168,8,2000,7.740175
8031,The Dark Knight Rises,9263,7,2012,6.921448
6218,Batman Begins,7511,7,2005,6.904127
1134,Batman Returns,1706,6,1992,5.846862
132,Batman Forever,1529,5,1995,5.054144
9024,Batman v Superman: Dawn of Justice,7189,5,2016,5.013943
1260,Batman & Robin,1447,4,1997,4.287233


# CF based recommendation system

In [64]:
# surprise reader API to read the dataset
reader = Reader()
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
data.split(n_folds=5)

svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.8956
MAE:  0.6914
------------
Fold 2
RMSE: 0.8986
MAE:  0.6887
------------
Fold 3
RMSE: 0.8857
MAE:  0.6832
------------
Fold 4
RMSE: 0.9014
MAE:  0.6943
------------
Fold 5
RMSE: 0.9011
MAE:  0.6927
------------
------------
Mean RMSE: 0.8965
Mean MAE : 0.6901
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.8956105535284721,
                             0.8985503412133158,
                             0.8857000628498031,
                             0.9013802619423751,
                             0.9011480458032547],
                            'mae': [0.6914251139963837,
                             0.6886817333838965,
                             0.6832464410334382,
                             0.6943465672474018,
                             0.6927442599807615]})

In [65]:
trainset = data.build_full_trainset()
svd.train(trainset)

In [66]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [67]:
svd.predict(1, 302)

Prediction(uid=1, iid=302, r_ui=None, est=2.79070100822545, details={'was_impossible': False})

In [68]:
svd.predict(1,1339)

Prediction(uid=1, iid=1339, r_ui=None, est=2.8616995853139375, details={'was_impossible': False})

# Hybrid recommendation system

In [69]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [70]:
id_map = pd.read_csv('../input/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')
indices_map = id_map.set_index('id')

In [71]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'release_date', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [72]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,release_date,id,est
522,Terminator 2: Judgment Day,4274.0,7.7,1991-07-01,280,3.211235
2014,Fantastic Planet,140.0,7.6,1973-05-01,16306,3.062090
974,Aliens,3282.0,7.7,1986-07-18,679,3.012815
1011,The Terminator,4208.0,7.4,1984-10-26,218,2.959696
1621,Darby O'Gill and the Little People,35.0,6.7,1959-06-29,18887,2.951909
1668,Return from Witch Mountain,38.0,5.6,1978-03-10,14822,2.852648
1376,Titanic,7770.0,7.5,1997-11-18,597,2.813935
922,The Abyss,822.0,7.1,1989-08-09,2756,2.813877
8401,Star Trek Into Darkness,4479.0,7.4,2013-05-05,54138,2.794536
8658,X-Men: Days of Future Past,6155.0,7.5,2014-05-15,127585,2.759219


# Testing with new data split

In [73]:
def get_count(tp, id):
    playcount_groupbyid = tp[[id]].groupby(id, as_index=False)
    count = playcount_groupbyid.size()
    return count

In [74]:
def filter_triplets(tp, min_uc=5, min_sc=0):
    # Only keep the triplets for items which were clicked on by at least min_sc users. 
    if min_sc > 0:
        itemcount = get_count(tp, 'movieId')
        tp = tp[tp['movieId'].isin(itemcount.index[itemcount >= min_sc])]
    
    # Only keep the triplets for users who clicked on at least min_uc items
    # After doing this, some of the items will have less than min_uc users, but should only be a small proportion
    if min_uc > 0:
        usercount = get_count(tp, 'userId')
        tp = tp[tp['userId'].isin(usercount.index[usercount >= min_uc])]
    
    # Update both usercount and itemcount after filtering
    usercount, itemcount = get_count(tp, 'userId'), get_count(tp, 'movieId') 
    return tp, usercount, itemcount

In [75]:
ratings, user_activity, item_popularity = filter_triplets(ratings)


In [76]:
sparsity = 1. * ratings.shape[0] / (user_activity.shape[0] * item_popularity.shape[0])

print("After filtering, there are %d watching events from %d users and %d movies (sparsity: %.3f%%)" % 
      (ratings.shape[0], user_activity.shape[0], item_popularity.shape[0], sparsity * 100))

After filtering, there are 100004 watching events from 671 users and 9066 movies (sparsity: 1.644%)


In [77]:
unique_uid = user_activity.index

np.random.seed(98765)
idx_perm = np.random.permutation(unique_uid.size)
unique_uid = unique_uid[idx_perm]

In [78]:
# create train/validation/test users
n_users = unique_uid.size
n_heldout_users = 100

tr_users = unique_uid[:(n_users - n_heldout_users)]
te_users = unique_uid[(n_users - n_heldout_users):]

In [79]:
df_train = ratings.loc[ratings['userId'].isin(tr_users)]
df_train.shape

(85236, 4)

In [80]:
unique_sid = pd.unique(df_train['movieId'])
len(unique_sid)

8575

In [81]:
df_test = ratings.loc[ratings['userId'].isin(te_users)]
df_test.shape

(14768, 4)

In [82]:
# surprise reader API to read the dataset
reader = Reader()
data1 = Dataset.load_from_df(df_test[['userId', 'movieId', 'rating']], reader)
data1.split(n_folds=5)

svd1 = SVD()
evaluate(svd1, data1, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.9105
MAE:  0.7042
------------
Fold 2
RMSE: 0.9180
MAE:  0.7045
------------
Fold 3
RMSE: 0.9052
MAE:  0.6960
------------
Fold 4
RMSE: 0.9150
MAE:  0.6987
------------
Fold 5
RMSE: 0.9056
MAE:  0.6992
------------
------------
Mean RMSE: 0.9109
Mean MAE : 0.7005
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.9104502415679713,
                             0.9179696544784116,
                             0.9051938498217603,
                             0.91504070186333,
                             0.9056134789620075],
                            'mae': [0.7041815311867572,
                             0.7044774914206297,
                             0.695973733404187,
                             0.6986747688851405,
                             0.6991886543103877]})

In [83]:
df_test.head()

,userId,movieId,rating,timestamp
147,4,10,4.0,949810645
148,4,34,5.0,949919556
149,4,112,5.0,949810582
150,4,141,5.0,949919681
151,4,153,4.0,949811346


In [84]:
id_map1 = id_map.reset_index()

In [85]:
id_map1.set_index('movieId')

,title,id
movieId,,
1,Toy Story,862.0
2,Jumanji,8844.0
3,Grumpier Old Men,15602.0
4,Waiting to Exhale,31357.0
5,Father of the Bride Part II,11862.0
6,Heat,949.0
7,Sabrina,11860.0
8,Tom and Huck,45325.0
9,Sudden Death,9091.0


In [86]:
def hybrid1(userId, movieId):
    title = id_map1.loc[movieId]['title']
    idx = indices[title]
    tmdbId = id_map1.loc[movieId]['id']
#     movie_id = id_map.loc[title]['movieId']
    movie_id = movieId
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:50]
    movie_indices = [i[0] for i in sim_scores]
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'release_date', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies

In [87]:
# df_final =  df_pred
df_pred = hybrid1(5,10)
df_final.append(df_pred)

NameError: name 'df_final' is not defined

In [88]:
df_final

NameError: name 'df_final' is not defined

In [89]:
df_test[df_test['userId']==4].sort_values('rating',ascending=False)

,userId,movieId,rating,timestamp
217,4,1243,5.0,949919519
267,4,2046,5.0,949810618
314,4,2723,5.0,949810582
276,4,2091,5.0,949896183
317,4,2788,5.0,949919399
232,4,1344,5.0,949919247
231,4,1334,5.0,949982274
318,4,2791,5.0,949811490
291,4,2161,5.0,949896070
320,4,2797,5.0,949896070


User Profile* 

In [90]:
df_user_ratings = ratings[ratings.userId==1]

In [91]:
df_user_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [92]:
df_user_ratings.rename(columns={'movieId':'id'},inplace=True)

In [93]:
df_user_ratings.head()

,userId,id,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [94]:
smd.tail()

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size,director,soup
9214,40952,False,NaN,8000000,[Drama],NaN,159550,tt0255313,en,The Last Brickmaker in America,...,7.0,1.0,2001,"[sidneypoitier, wendycrewson, jayo.sanders]","[{'credit_id': '544475aac3a36819fb000578', 'de...",[friendship],7,2,"[greggchampion, greggchampion, greggchampion]",friendship sidneypoitier wendycrewson jayo.san...
9215,41172,False,NaN,1000000,"[Thriller, Romance]",NaN,392572,tt5165344,hi,रुस्तम,...,7.3,25.0,2016,"[akshaykumar, ileanad'cruz, eshagupta]","[{'credit_id': '5951baf692514129c4016600', 'de...",[bollywood],14,16,"[tinusureshdesai, tinusureshdesai, tinusureshd...",bollywood akshaykumar ileanad'cruz eshagupta t...
9216,41225,False,NaN,15050000,"[Adventure, Drama, History, Romance]",NaN,402672,tt3859980,hi,Mohenjo Daro,...,6.7,26.0,2016,"[hrithikroshan, poojahegde, kabirbedi]","[{'credit_id': '57cd5d3592514179d50018e8', 'de...",[bollywood],12,16,"[ashutoshgowariker, ashutoshgowariker, ashutos...",bollywood hrithikroshan poojahegde kabirbedi a...
9217,41391,False,NaN,15000000,"[Action, Adventure, Drama, Horror, Science Fic...",NaN,315011,tt4262980,ja,シン・ゴジラ,...,6.6,152.0,2016,"[hirokihasegawa, yutakatakenouchi, satomiishih...","[{'credit_id': '560892fa92514177550018b2', 'de...","[monster, godzilla, giantmonst, destruct, kaiju]",49,27,"[hideakianno, hideakianno, hideakianno]",monster godzilla giantmonst destruct kaiju hir...
9218,41669,False,NaN,0,"[Documentary, Music]",http://www.thebeatlesliveproject.com/,391698,tt2531318,en,The Beatles: Eight Days a Week - The Touring Y...,...,7.6,92.0,2016,"[paulmccartney, ringostarr, johnlennon]","[{'credit_id': '57057c6cc3a3680dca000285', 'de...","[music, documentari]",28,7,"[ronhoward, ronhoward, ronhoward]",music documentari paulmccartney ringostarr joh...


In [95]:
df_user_data_with_tags = smd.reset_index().merge(df_user_ratings, on='id')

In [96]:
df_user_data_with_tags.head(10)

,level_0,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,...,cast,crew,keywords,cast_size,crew_size,director,soup,userId,rating,timestamp
0,1937,2317,False,"{'id': 1575, 'name': 'Rocky Collection', 'post...",17000000,[Drama],NaN,1371,tt0084602,en,...,"[sylvesterstallone, taliashire, burtyoung]","[{'credit_id': '52fe42f0c3a36847f802e4d1', 'de...","[transport, trainer, boxer, dyinganddeath, box...",15,20,"[sylvesterstallone, sylvesterstallone, sylvest...",transport trainer boxer dyinganddeath boxingma...,1,2.5,1260759135
1,2181,2618,False,"{'id': 2806, 'name': 'American Pie Collection'...",11000000,"[Comedy, Romance]",NaN,2105,tt0163651,en,...,"[jasonbiggs, chrisklein, thomasiannicholas]","[{'credit_id': '55722baac3a368368e0023cc', 'de...","[graduat, innoc, comingofag, teenag, highschoo...",43,86,"[chrisweitz, chrisweitz, chrisweitz]",graduat innoc comingofag teenag highschoolstud...,1,4.0,1260759139
2,2684,3240,False,NaN,0,"[Comedy, Drama, Romance]",http://www.crownintlpictures.com/lntitles.html,2193,tt0085980,en,...,"[carenkaye, mattlattanzi, kevinmccarthy]","[{'credit_id': '52fe433fc3a36847f8045b41', 'de...","[learningandteach, tutor, olderwomanyoungerman...",18,11,"[georgebowers, georgebowers, georgebowers]",learningandteach tutor olderwomanyoungermanrel...,1,2.0,1260759198
3,3739,4645,False,NaN,22000000,[Comedy],NaN,2294,tt0261392,en,...,"[kevinsmith, jasonmewes, benaffleck]","[{'credit_id': '556a356f9251415408000e78', 'de...","[filmmak, jayandsilentbob, indianajonesspoofsc...",29,117,"[kevinsmith, kevinsmith, kevinsmith]",filmmak jayandsilentbob indianajonesspoofscen ...,1,2.0,1260759108
4,5721,8393,False,NaN,546883,"[Drama, History]",NaN,1405,tt0015881,en,...,"[zasupitts, gibsongowland, jeanhersholt]","[{'credit_id': '52fe42f3c3a36847f802f33d', 'de...","[gold, lotteri, marriag, murder, money, traged...",24,26,"[erichvonstroheim, erichvonstroheim, erichvons...",gold lotteri marriag murder money tragedi pove...,1,1.0,1260759203


In [97]:
df_user_data_with_tags.shape

(5, 37)

In [98]:
df_user_data_with_tags['index'].values

array([2317, 2618, 3240, 4645, 8393])

In [99]:
user_profile = np.dot(tfidf_matrix[df_user_data_with_tags['index'].values].toarray().T, df_user_data_with_tags['rating'].values)

In [100]:
tfidf_matrix[df_user_data_with_tags['index'].values]

<5x268124 sparse matrix of type '<class 'numpy.float64'>'
	with 338 stored elements in Compressed Sparse Row format>

In [101]:
user_profile

array([0., 0., 0., ..., 0., 0., 0.])

In [102]:
from sklearn.metrics.pairwise import cosine_similarity

In [103]:
C = cosine_similarity(np.atleast_2d(user_profile), tfidf_matrix)

In [104]:
R = np.argsort(C)[:, ::-1]

In [105]:
R

array([[2618, 2317, 4645, ..., 4776, 4769,    0]])

In [106]:
recommendations = [i for i in R[0] if i not in df_user_data_with_tags['index'].values]

In [107]:
smd['title'][recommendations].head(10)

2055          Lock, Stock and Two Smoking Barrels
428                                    Kalifornia
9012                     Sharknado 3: Oh Hell No!
8014    The Twilight Saga: Breaking Dawn - Part 1
658                                      Daylight
3085               Phantasm III: Lord of the Dead
7127                                     The Wave
5024                                The Navigator
4622                               Fahrenheit 451
5359                                  From Beyond
Name: title, dtype: object